In [56]:
from tqdm import tqdm
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Sequential

from src.const import AUDIO_PATH, MAIN_LABELS, BATCH_SIZE, VALIDATION_SPLIT, SEED

from src.preprocess import load_and_preprocess_for_binary_task

### Unknown vs known task

In [ ]:
train_ds, val_ds = load_and_preprocess_for_binary_task(plot_samples = True)

In [51]:
model = Sequential()
model.add(Bidirectional(LSTM(128), merge_mode="sum"))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(Dropout(0.2))
model.add(Dense(units=2, activation="softmax", kernel_regularizer=l2(0.01)))

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [52]:
# input_shape=(63,128)
# model = tf.keras.Sequential()
# model.add(LSTM(128,input_shape=input_shape))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(11, activation='softmax'))
# model.summary()

In [53]:
known_vs_unknown_train_ds = train_ds.cache().shuffle(100).batch(128).prefetch(tf.data.AUTOTUNE)
known_vs_unknown_val_ds = val_ds.cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
history = model.fit(known_vs_unknown_train_ds, epochs=50, validation_data=known_vs_unknown_val_ds, callbacks=[es])